# Non-Councurrency

In [ ]:
import time

def fetch_data():
    print("Hi")
    time.sleep(3)
    print("Bye")

def main():
    start_time = time.perf_counter()    # Start the timer

    # Execute tasks sequentially
    for _ in range(2):
        fetch_data()
    
    end_time = time.perf_counter()      # End the timer
    print(f"Total time taken: {end_time - start_time:.2f} seconds")

main()

Hi
Bye
Hi
Bye
Total time taken: 6.00 seconds


- The program runs sequentially and takes 6 seconds to complete

# Non-Councurrency (Threading)

In [3]:
import threading
import time

def fetch_data():
    print(f"Start fetching data on thread: {threading.get_ident()}")
    time.sleep(3)
    print(f"Finished fetching data on thread: {threading.get_ident()}")

def main():
    start_time = time.perf_counter()    # Start the timer

    # Execute tasks sequentially
    for _ in range(2):
        fetch_data()
    
    end_time = time.perf_counter()      # End the timer
    print(f"Total time taken: {end_time - start_time:.2f} seconds")

main()

Start fetching data on thread: 5952
Finished fetching data on thread: 5952
Start fetching data on thread: 5952
Finished fetching data on thread: 5952
Total time taken: 6.00 seconds


- it still running sequentially, this version just import thread but not actually create different thread to run so it still *non-concurrency* 

# Concurrency (Threading)

In [4]:
import threading
import time

def fetch_data():
    print(f"Start fetching data on thread: {threading.get_ident()}")
    time.sleep(3)
    print(f"Finished fetching data on thread: {threading.get_ident()}")

def main():
    start_time = time.perf_counter()

    threads = []
    for _ in range(2):
        t = threading.Thread(target=fetch_data)
        t.start()
        threads.append(t)

    for t in threads:
        t.join()  # Wait for all threads to finish

    end_time = time.perf_counter()
    print(f"Total time taken: {end_time - start_time:.2f} seconds")

main()


Start fetching data on thread: 13472
Start fetching data on thread: 3492
Finished fetching data on thread: 3492
Finished fetching data on thread: 13472
Total time taken: 3.03 seconds


- Now it actually running concurrency, because i actually create different thread to run the tasks

# Concurrency (Async/await)

In [ ]:
import asyncio
import time

async def fetch_data():
    print("Hi")
    await asyncio.sleep(3)
    print("Bye")

async def main():
    start_time = time.perf_counter()    # Start the timer

    # Create a list of tasks
    tasks = [fetch_data() for _ in range(2)]

    # Run tasks concurrently
    await asyncio.gather(*tasks)

    end_time = time.perf_counter()      # End the timer
    print(f"TOtal time taken: {end_time - start_time:.2f} seconds")

# asyncio.run(main()) -> this will cause error: RuntimeError: asyncio.run() cannot be called from a running event loop
# because The notebook kernel is already running an event loop — and asyncio.run() tries to start a new one, which causes the conflict.

await main() # this only works in environments that support top-level await (Jupyter, VSCode Notebooks, etc).

Hi
Hi
Bye
Bye
TOtal time taken: 2.99 seconds


- `Event Loop` là trái tim của `asyncio`
    - Nó là một `vòng lặp` chạy liên tục để kiểm tra và thực hiện các `tác vụ bất đồng bộ (coroutines)` đã sẵn sàng
    - Khi một tác vụ đang chờ (ví dụ: `await asyncio.sleep(3)`), event loop có thể chuyển sang xử lý tác vụ khác
    - Đây là cách python mô phỏng `concurrency` mà không cần thread hay process riêng biệt

- `await` dùng trong hàm `async def` để:
    - `tạm dừng` quá trình chạy hàm đó
    - `nhường quyền điều khiển` lại cho `event loop`
    - để event loop có thể xử lý công việc khác trong thời gian chờ
    - `await asyncio.sleep(3)`:
        - không chặn (blocking) chương trình 3 giây
        - mà nhường chỗ cho tác vụ khác chạy trong thời gian đó
        - chỉ dùng được trong hàm `async def`

- `asyncio.run()` là cách `chính thống` để chạy một chương trình bất đồng bộ từ đầu:
    - nó sẽ tự tạo 1 event loop mới
    - chạy main() `coroutine`
    - và đóng lại event loop khi hoàn thành
    - chỉ dùng được trong môi trường chưa có event loop đang chạy

- Khi chạy `asyncio.run()` trong `.ipynb` file lỗi vì:
    - `Event loop đã chạy sẵn` do notebook cần chạy bất đồng bộ (jupyter dùng Tornado - một async web framework)
    - Nếu đang gọi `asyncio.run(main())`, lúc này đang cố tạo event loop mới, nhưng python không cho phép -> lỗi
    - chạy trực tiếp `await main()` trong `.ipynb file`:
        - khi chạy một `cell`, jupyter tự bao nó trong một `async def wrapper()` phía sau hậu trường
        - nhờ đó, có thể dùng `await` trực tiếp như thể trong 1 `async def` 

# Refs:
- https://drive.google.com/drive/folders/1PQqHoLs6OGZxuDlyMTIfTZ5vXRjQkVPf 